In [1]:
def _calc_output_size(input_size, padding, dilation, kernel_size, stride):
    input_size_padded = input_size + 2 * padding
    kernel_dilated = (kernel_size - 1) * (dilation - 1) + kernel_size
    output_size = (input_size_padded - kernel_dilated) // stride + 1
    return output_size

In [17]:

# layer 1
in_features = 64
output_size = _calc_output_size(in_features, padding=3, dilation=1, kernel_size=7, stride=2)
print('conv1\t\t',in_features, output_size,'padding=3, dilation=1, kernel_size=7, stride=2')

# maxpool
in_features = output_size
output_size = _calc_output_size(in_features, padding=1, dilation=1, kernel_size=3, stride=2)
print('maxpool\t\t',in_features, output_size,'padding=0, dilation=1, kernel_size=3, stride=2')

# layer 2
in_features = output_size
output_size = _calc_output_size(in_features, padding=1, dilation=1, kernel_size=3, stride=2)
print('conv2..1\t',in_features, output_size, 'padding=1, dilation=1, kernel_size=3, stride=2')

# layer 3
in_features = output_size
output_size = _calc_output_size(in_features, padding=1, dilation=1, kernel_size=3, stride=1)
print('conv2..2\t',in_features, output_size, 'padding=1, dilation=1, kernel_size=3, stride=1')

# layer 4
in_features = output_size
output_size = _calc_output_size(in_features, padding=1, dilation=1, kernel_size=3, stride=1)
print('conv2..3\t',in_features, output_size,'padding=1, dilation=1, kernel_size=3, stride=1')

# layer 5
in_features = output_size
output_size = _calc_output_size(in_features, padding=1, dilation=1, kernel_size=3, stride=1)
print('conv2..4\t',in_features, output_size,'padding=1, dilation=1, kernel_size=3, stride=1')

# layer 6


conv1		 64 32 padding=3, dilation=1, kernel_size=7, stride=2
maxpool		 32 16 padding=0, dilation=1, kernel_size=3, stride=2
conv2..1	 16 8 padding=1, dilation=1, kernel_size=3, stride=2
conv2..2	 8 8 padding=1, dilation=1, kernel_size=3, stride=1
conv2..3	 8 8 padding=1, dilation=1, kernel_size=3, stride=1
conv2..4	 8 8 padding=1, dilation=1, kernel_size=3, stride=1


In [5]:
import torch
import torch.nn as nn

In [68]:
#https://towardsdatascience.com/residual-network-implementing-resnet-a7da63c7b278
class ResidualBlock(nn.Module):

    def __init__(self, in_channels, out_channels):
        super().__init__()
        
        self.expansion= 1 
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.stride = 2 if in_channels != out_channels else 1

        self.blocks = nn.Sequential(
        
            # first conv layer
            nn.Conv2d(self.in_channels, self.out_channels, kernel_size=3, stride=self.stride, padding=1, bias=False),
            nn.BatchNorm2d(self.out_channels),
            nn.ReLU(inplace=True),

            # second conv layer
            nn.Conv2d(self.out_channels, self.out_channels, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(self.out_channels))

        # shortcut
        if self.in_channels == self.out_channels:
            self.shortcut = nn.Identity()
        else:
#             self.shortcut = nn.Identity()
            self.shortcut = nn.Sequential(
                nn.Conv2d(self.in_channels, self.out_channels, kernel_size=1, stride=self.stride, bias=False),
                nn.BatchNorm2d(self.out_channels)
            )
        
        self.activate = nn.ReLU(inplace=True)

    def forward(self, x):
        # blocks
        out = self.blocks(x)

        # shortcut
        shortcut = self.shortcut(x)

        # combine
        out = self.activate(out + shortcut)

        return out

In [69]:
block = ResidualBlock(64,128)
block


ResidualBlock(
  (blocks): Sequential(
    (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (4): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (shortcut): Sequential(
    (0): Conv2d(64, 128, kernel_size=(1, 1), stride=(2, 2), bias=False)
    (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (activate): ReLU(inplace=True)
)

In [57]:
dummy = torch.ones([1,64,48,48])
block(dummy).shape

torch.Size([1, 128, 24, 24])

In [70]:
class Resnet18(nn.Module):
    def __init__(self, in_features, num_classes):
        super().__init__()

        self.layers = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=64, kernel_size=7, stride=2, padding=3, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(),

            nn.MaxPool2d(kernel_size=3, stride=2, padding=1),

            # layer 1
            ResidualBlock(64, 64),
            ResidualBlock(64, 64),

            ResidualBlock(64, 128),
            ResidualBlock(128, 128),

            ResidualBlock(128, 256),
            ResidualBlock(256, 256),

            ResidualBlock(256, 512),
            ResidualBlock(512, 512),

            nn.AdaptiveAvgPool2d((1, 1)),
            nn.Flatten(),
        )
        self.linear = nn.Linear(512, num_classes)

    def forward(self, x, return_embedding=False):
        embedding = self.layers(x)

        if return_embedding:
            return embedding
        else:
            return self.linear(embedding)

In [71]:
r= Resnet18(3,1000)
r

Resnet18(
  (layers): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (4): ResidualBlock(
      (blocks): Sequential(
        (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(inplace=True)
        (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (shortcut): Identity()
      (activate): ReLU(inplace=True)
    )
    (5): ResidualBlock(
      (blocks): Sequential(
        (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (1): Bat

In [72]:
from torchsummary import summary
model = Resnet18(3, 1000)
summary(model, (3, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,408
       BatchNorm2d-2         [-1, 64, 112, 112]             128
              ReLU-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 56, 56]               0
            Conv2d-5           [-1, 64, 56, 56]          36,864
       BatchNorm2d-6           [-1, 64, 56, 56]             128
              ReLU-7           [-1, 64, 56, 56]               0
            Conv2d-8           [-1, 64, 56, 56]          36,864
       BatchNorm2d-9           [-1, 64, 56, 56]             128
         Identity-10           [-1, 64, 56, 56]               0
             ReLU-11           [-1, 64, 56, 56]               0
    ResidualBlock-12           [-1, 64, 56, 56]               0
           Conv2d-13           [-1, 64, 56, 56]          36,864
      BatchNorm2d-14           [-1, 64,